In [ ]:
# A faire tourner que si vous n'avez pas déjà installé les packages nécessaires
!pip install gensim==3.8.3
!pip install unidecode
!pip install torch
!pip install sentencepiece
!pip install transformers
!pip install bs4
!pip install networkx

In [ ]:
from joblib.parallel import cpu_count
import pandas as pd
import numpy as np
import torch
import pickle
import time
import functools
import operator
from joblib import Parallel,delayed
from functools import partial
import sentencepiece as spm 
import psutil
from tqdm import tqdm
import pickle
import os
import re
from transformers.utils.dummy_pt_objects import CamembertModel
from transformers.utils.dummy_sentencepiece_objects import CamembertTokenizer
from unidecode import unidecode
from bs4 import BeautifulSoup
import gensim
import sys
import gc
import fats

In [98]:
N=[f for f in os.listdir() if (f.split('.')[-1]=='pickle') and ('sortie' in f)]
N=N[1:]
N

['Net_sortie_3.pickle',
 'TRW_sortie.pickle',
 'TRB_sortie_finale.pickle',
 'SMHA_sortie_3.pickle',
 'L3_sortie.pickle',
 'BSR_sortie.pickle',
 'Simple_sortie_finale.pickle',
 'RS_sortie.pickle',
 'Multi_sortie_finale.pickle']

In [58]:
index_2=pickle.load(open('index_2.pickle','rb'))
trace=pickle.load(open('trace_test.pickle','rb'))
score=pickle.load(open('score.pickle','rb'))
score=[score[i] for i in index_2]
assert len(score)==len(trace)
s=Parallel(5)(delayed(fats.make_new_paragraphes)(i,j) for i,j in zip(score,trace))
s=functools.reduce(operator.iconcat,s,[])
P=pickle.load(open('Paragraphes_.pickle','rb'))
assert len(P)==len(s)

In [79]:
for n in N:
    SMHA=pickle.load(open(n,'rb'))
    try:
        Parallel(5)(delayed(fats.make_new_sortie)(i,j) for i,j in zip(s,SMHA))
    except:
        print(n)

L3_sortie_.pickle
Net_sortie_3.pickle
SMHA_sortie_3.pickle
BSR_sortie.pickle
RS_sortie.pickle


In [84]:
np.mean([len(i) for i in s])/2

9.714294718265473

In [82]:
BSR=pickle.load(open('BSR_sortie.pickle','rb'))
np.sum([1 for i in range(len(s)) if len(s[i])!=len(BSR[i])])

22152

In [75]:
sortie=pickle.load(open(N[3],'rb'))
sortie[0],s[0]

([2, 3],
 tensor([0.1555, 0.0826, 0.1421, 0.0972, 0.1317, 0.0945, 0.1363, 0.0868, 0.0838,
         0.0979, 0.0958, 0.0903, 0.0905, 0.0478, 0.0964, 0.1024, 0.0928]))

In [91]:
L3=pickle.load(open(N[0],'rb'))
l=[i for i in range(len(s)) if len(s[i])==1]
for i in l:
    L3[i]=L3[i][:1]

    pickle.dump(L3,open('L3_sortie.pickle','wb'))
L3[l[0]]

tensor([0])

In [105]:
Net=pickle.load(open('Net_sortie_3.pickle','rb'))
dico=pickle.load(open('dico_comparaison.pickle','rb'))
dico.keys()

dict_keys(['phrase', 'score', 'erreur', 'trace'])

In [106]:
len(dico['erreur'])

5034

In [118]:
np.sum([len(dico_sortie['Simple_sortie_finale'][i])==len(s[i]) for i in range(len(s))])

23799

In [102]:
erreur=[]
B=pickle.load(open('BSR_sortie.pickle','rb'))
BSR=[]
for i in range(len(s)):
    try:
        BSR.append(fats.make_new_sortie(s[i],B[i]))
    except:
        erreur.append(i)
erreur

[201]

In [97]:
dico_sortie={}
for n in N:
    sortie=pickle.load(open(n,'rb'))
    try:
        dico_sortie[n.split('.')[0]]=Parallel(5)(delayed(fats.make_new_sortie)(i,j) for i,j in zip(s,sortie))
    except:
        continue
dico_sortie.keys()

UnpicklingError: invalid load key, '\x00'.

In [ ]:
def comparaison(f,j=2):
    dico=pickle.load(open(dico,'rb'))
    l=[i for i in range(len(dico['score'])) if len(dico['phrase'][i])==len(dico['score'][i]) ]
    score_vrai=[dico['score'][i] for i in l]
#     fichiers=[i for i in os.listdir() if (name in i) and ('.pickle' in i)]
    sortie=pickle.load(open(f,'rb'))
    if len(sortie)==23799:
        simple=[sortie[i] for i in range(len(sortie)) if i not in dico['erreur']]
        simple2=[simple[i] for i in l]
        simple3=Parallel(j)(delayed(make_new_sortie)(i,j) for i,j in zip(score_vrai,simple2))
    
    assert len(simple3)==len(score_vrai)
        
    F1=F1_score()
    tp=[]
    fp=[]
    fn=[]
    p=[]
    r=[]
    f=[]
    
    for i in tqdm(range(len(simple3))):
        tp.append(F1.true_positive_mean(simple3[i],score_vrai[i]))
        fp.append(F1.false_positive_mean(simple3[i],score_vrai[i]))
        fn.append(F1.false_negative_mean(simple3[i],score_vrai[i]))
        p.append(F1.precision(simple3[i],score_vrai[i]))
        r.append(F1.recall(simple3[i],score_vrai[i]))
        f.append(F1(simple3[i],score_vrai[i]))
        
    mtp=torch.mean(torch.tensor(tp))
    mfp=torch.mean(torch.tensor(fp))
    mfn=torch.mean(torch.tensor(fn))
    mp=torch.mean(torch.tensor(p))
    mr=torch.mean(torch.tensor(r))
    mf=torch.mean(torch.tensor(f))
    resultat=[mtp,mfp,mfn,mp,mr,mf]
    
    return resultat